# Tool 기반 투자 분석 Agent (ReAct 패턴)

## 주요 기능:
- ✅ **LangChain Tool 프레임워크** 활용
- ✅ **ReAct 패턴** (Reasoning + Acting)
- ✅ **4개의 투자 분석 도구**
- ✅ **LangGraph prebuilt agent** 사용
- ✅ **자동 도구 선택 및 실행**

In [ ]:
# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

## 1. 투자 분석 도구 가져오기

In [ ]:
import sys
sys.path.append('..')

from python.models.tools import (
    AVAILABLE_TOOLS,
    search_web,
    get_stock_price,
    calculate_moving_average,
    get_company_info,
    ToolAgentState
)

print(f"✅ {len(AVAILABLE_TOOLS)}개의 도구 로드 완료:")
for t in AVAILABLE_TOOLS:
    print(f"  - {t.name}: {t.description}")

## 2. ReAct Agent 생성

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

# LLM 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 시스템 프롬프트
system_prompt = """당신은 전문 주식 투자 분석가입니다.

**역할**:
- 사용자의 투자 관련 질문에 데이터 기반으로 답변
- 필요한 경우 제공된 도구를 사용하여 정보 수집
- 실시간 주가, 기업 정보, 뉴스 등을 활용한 분석

**사용 가능한 도구**:
1. search_web: 웹에서 최신 뉴스 및 정보 검색
2. get_stock_price: 특정 주식의 가격 정보 조회
3. calculate_moving_average: 기술적 분석 (이동평균선)
4. get_company_info: 기업 기본 정보 및 재무 지표

**답변 원칙**:
- 구체적인 데이터와 출처를 제시
- 불확실한 정보는 명시
- 투자 결정은 사용자의 몫임을 강조
"""

# ReAct Agent 생성
agent = create_react_agent(
    llm,
    AVAILABLE_TOOLS,
    prompt=system_prompt  # state_modifier 대신 prompt 사용
)

print("✅ ReAct Agent 생성 완료")
print(f"   도구 개수: {len(AVAILABLE_TOOLS)}")
print(f"   LLM 모델: {llm.model_name}")

## 3. 그래프 구조 시각화

In [ ]:
from IPython.display import Image, display

try:
    display(Image(agent.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"그래프 시각화 실패: {e}")
    print("\n텍스트 구조:")
    print("START → __start__ → agent → tools → agent → __end__ → END")

## 4. Agent 실행 헬퍼 함수

In [ ]:
def run_agent(query: str, verbose: bool = True):
    """
    Agent를 실행하고 결과를 출력합니다.
    
    Args:
        query: 사용자 질문
        verbose: 실행 과정 출력 여부
    """
    print(f"\n{'='*70}")
    print(f"질문: {query}")
    print(f"{'='*70}\n")
    
    messages = [{"role": "user", "content": query}]
    
    for chunk in agent.stream({"messages": messages}, stream_mode="values"):
        if verbose:
            last_message = chunk["messages"][-1]
            
            if hasattr(last_message, 'content') and last_message.content:
                print(f"[{last_message.__class__.__name__}]")
                print(last_message.content)
                print()
            elif hasattr(last_message, 'tool_calls') and last_message.tool_calls:
                for tc in last_message.tool_calls:
                    print(f"🔧 도구 호출: {tc['name']}")
                    print(f"   입력: {tc['args']}")
                print()
    
    # 최종 답변
    final_message = chunk["messages"][-1]
    print(f"\n{'='*70}")
    print("최종 답변:")
    print(f"{'='*70}")
    print(final_message.content)
    print(f"{'='*70}\n")
    
    return chunk

print("✅ 헬퍼 함수 정의 완료")

## 5. 테스트 1: 간단한 주가 조회

In [ ]:
result1 = run_agent("삼성전자 현재 주가 알려줘")

## 6. 테스트 2: 복합 분석 (여러 도구 사용)

ReAct 패턴에 따라 Agent가 여러 도구를 순차적으로 사용합니다.

In [ ]:
result2 = run_agent(
    "삼성전자(005930.KS)의 현재 주가와 20일 이동평균을 비교하고, "
    "최근 관련 뉴스도 검색해서 투자 의견을 제시해줘"
)

## 7. 테스트 3: 미국 주식 분석

In [ ]:
result3 = run_agent(
    "Apple(AAPL) 주식의 기업 정보와 최근 3개월 주가 추이를 분석해줘"
)

## 8. 불변 속성 검증 예시

Agent 상태의 불변 속성을 런타임에 검증할 수 있습니다.

In [ ]:
from python.models.tools import ToolHistory, ToolExecution

# 초기 상태 생성
state = ToolAgentState.initial_state("테스트 질문", max_calls=5)

# 도구 실행 시뮬레이션
exec1 = ToolExecution(
    tool_name="search_web",
    arguments={"query": "test"},
    result="테스트 결과",
    call_id="call_1"
)
state.tool_history.add_execution(exec1)
state.current_iteration += 1

# 불변 속성 검증
print("불변 속성 검증:")
print(f"  도구 호출 횟수 제한: {state.tool_history.total_calls <= state.max_tool_calls}")
print(f"  히스토리 일관성: {len(state.tool_history.executions) == state.tool_history.total_calls}")
print(f"  전체 검증: {state.verify_invariants()}")
print(f"\n현재 상태:")
print(f"  도구 호출 횟수: {state.tool_history.total_calls}/{state.max_tool_calls}")
print(f"  더 호출 가능: {state.can_call_more_tools()}")

## 9. 실습 문제

아래 10개의 문제를 순서대로 풀어보세요. 각 문제는 점점 어려워집니다.

### 문제 1: 도구 개수 확인하기

**목표**: `AVAILABLE_TOOLS` 리스트에 몇 개의 도구가 있는지 출력하세요.

**힌트**: `len()` 함수를 사용하세요.

In [ ]:
# TODO: AVAILABLE_TOOLS의 개수를 출력하세요
tool_count = ___  # 빈칸을 채우세요
print(f"사용 가능한 도구 개수: {tool_count}")

# 기대 결과: 사용 가능한 도구 개수: 4

### 문제 2: 도구 이름 출력하기

**목표**: 모든 도구의 이름을 for 루프를 사용해서 출력하세요.

**힌트**: `tool.name`으로 도구 이름에 접근할 수 있습니다.

In [ ]:
# TODO: 모든 도구의 이름을 출력하세요
for tool in ___:  # 빈칸을 채우세요
    print(f"- {___}")  # 빈칸을 채우세요

# 기대 결과:
# - search_web
# - get_stock_price
# - calculate_moving_average
# - get_company_info

### 문제 3: 특정 도구 찾기

**목표**: `find_tool()` 함수를 사용해서 "get_stock_price" 도구를 찾고, 그 설명을 출력하세요.

**힌트**: `from python.models.tools import find_tool`로 함수를 가져오세요.

In [ ]:
from python.models.tools import find_tool

# TODO: get_stock_price 도구를 찾아서 설명을 출력하세요
tool = find_tool(___)  # 빈칸을 채우세요
if tool:
    print(f"도구 설명: {tool.description}")
else:
    print("도구를 찾을 수 없습니다")

# 기대 결과: 도구 설명: 특정 주식의 가격 정보를 조회합니다...

### 문제 4: yfinance로 주가 가져오기

**목표**: `yfinance`를 사용해서 Apple(AAPL)의 최근 1개월 주가 데이터를 가져오세요.

**힌트**: `yf.Ticker(티커).history(period=기간)`를 사용하세요.

In [ ]:
import yfinance as yf

# TODO: Apple 주가 데이터 가져오기
ticker = ___  # Apple 티커 심볼 입력
stock = yf.Ticker(ticker)
hist = stock.history(period=___)  # 1개월: "1mo"

print(f"데이터 행 개수: {len(hist)}")
print(f"최근 종가: {hist['Close'].iloc[-1]:.2f}")

# 기대 결과: 20~30개 정도의 데이터 행과 최근 종가

### 문제 5: 최고가와 최저가 찾기

**목표**: Apple(AAPL)의 최근 1개월 데이터에서 최고가와 최저가를 찾으세요.

**힌트**: `.max()`와 `.min()` 메서드를 사용하세요.

In [ ]:
import yfinance as yf

stock = yf.Ticker("AAPL")
hist = stock.history(period="1mo")

# TODO: 최고가와 최저가 찾기
high = hist['High'].___()  # 빈칸을 채우세요
low = hist['Low'].___()    # 빈칸을 채우세요

print(f"최근 1개월 최고가: {high:.2f}")
print(f"최근 1개월 최저가: {low:.2f}")
print(f"변동폭: {high - low:.2f}")

# 기대 결과: 최고가, 최저가, 변동폭이 출력됨

### 문제 6: 이동평균선 계산하기

**목표**: Apple(AAPL)의 5일 이동평균을 계산하세요.

**힌트**: `.rolling(window=기간).mean()`을 사용하세요.

In [ ]:
import yfinance as yf

stock = yf.Ticker("AAPL")
hist = stock.history(period="1mo")

# TODO: 5일 이동평균 계산
ma5 = hist['Close'].rolling(window=___).mean()  # 빈칸을 채우세요

print(f"최근 5일 이동평균: {ma5.iloc[-1]:.2f}")
print(f"현재가: {hist['Close'].iloc[-1]:.2f}")

# 현재가가 이동평균보다 높으면 상승 추세
if hist['Close'].iloc[-1] > ma5.iloc[-1]:
    print("신호: 상승 추세")
else:
    print("신호: 하락 추세")

# 기대 결과: 5일 이동평균과 현재가, 추세 신호

### 문제 7: 간단한 도구 함수 만들기

**목표**: 주식 티커를 받아서 현재가를 반환하는 함수를 만드세요.

**힌트**: 위에서 배운 yfinance 사용법을 함수로 만드세요.

In [ ]:
import yfinance as yf

def get_current_price(ticker: str) -> float:
    """주식의 현재가를 반환합니다."""
    # TODO: 함수를 완성하세요
    stock = yf.Ticker(___)  # 빈칸을 채우세요
    hist = stock.history(period="1d")
    return hist['Close'].iloc[-1]

# 테스트
price = get_current_price("AAPL")
print(f"Apple 현재가: ${price:.2f}")

price2 = get_current_price("MSFT")
print(f"Microsoft 현재가: ${price2:.2f}")

# 기대 결과: Apple과 Microsoft의 현재가 출력

### 문제 8: @tool 데코레이터 사용하기

**목표**: 위에서 만든 함수에 `@tool` 데코레이터를 추가하여 Agent가 사용할 수 있는 도구로 만드세요.

**힌트**: `from langchain_core.tools import tool`과 `Annotated`를 사용하세요.

In [ ]:
from langchain_core.tools import tool
from typing import Annotated
import yfinance as yf

# TODO: @tool 데코레이터를 추가하세요
___  # 빈칸을 채우세요
def get_simple_price(ticker: Annotated[str, "주식 티커 심볼"]) -> str:
    """주식의 현재가를 조회합니다."""
    stock = yf.Ticker(ticker)
    hist = stock.history(period="1d")
    price = hist['Close'].iloc[-1]
    return f"{ticker} 현재가: ${price:.2f}"

# 테스트
print(get_simple_price.name)  # 도구 이름 확인
print(get_simple_price.description)  # 도구 설명 확인
result = get_simple_price.invoke({"ticker": "AAPL"})
print(result)

# 기대 결과: 도구 이름, 설명, Apple 현재가

### 문제 9: ToolAgentState 사용하기

**목표**: `ToolAgentState`를 생성하고, 도구 실행 기록을 추가한 후, 불변 속성을 검증하세요.

**힌트**: `initial_state()`, `ToolExecution`, `add_execution()`, `verify_invariants()`를 사용하세요.

In [ ]:
from python.models.tools import ToolAgentState, ToolHistory, ToolExecution

# TODO: 초기 상태 생성 (최대 3번 호출 가능)
state = ToolAgentState.initial_state(
    query="테스트 질문",
    max_calls=___  # 빈칸을 채우세요
)

# TODO: 첫 번째 도구 실행 기록 추가
exec1 = ToolExecution(
    tool_name=___,  # "get_stock_price" 입력
    arguments={"ticker": "AAPL"},
    result="Apple 현재가: $150.00",
    call_id="call_1"
)
state.tool_history.add_execution(___)  # 빈칸을 채우세요

# TODO: 두 번째 도구 실행 기록 추가
exec2 = ToolExecution(
    tool_name="search_web",
    arguments={"query": "Apple news"},
    result="최신 뉴스...",
    call_id="call_2"
)
state.tool_history.add_execution(exec2)

# 상태 확인
print(f"도구 호출 횟수: {state.tool_history.total_calls}")
print(f"더 호출 가능: {state.can_call_more_tools()}")
print(f"불변 속성 검증: {state.verify_invariants()}")

# 기대 결과: 호출 횟수 2, 더 호출 가능 True, 검증 True

### 문제 10: 나만의 Agent 질문 만들기

**목표**: Agent에게 물어볼 창의적인 투자 분석 질문을 만들고 실행하세요.

**힌트**:
- 여러 도구를 사용하도록 복합적인 질문을 만드세요.
- 예: "회사 정보 + 주가 + 뉴스" 조합

**추천 질문 예시**:
- "Tesla(TSLA)의 기업 정보를 조회하고, 현재가와 50일 이동평균을 비교해줘"
- "Microsoft(MSFT)의 최근 주가와 AI 관련 뉴스를 검색해서 투자 의견 알려줘"
- "Netflix(NFLX)의 재무지표와 최근 실적 뉴스를 분석해줘"

In [ ]:
# TODO: 나만의 질문을 만들어보세요
my_question = """
___  # 여기에 질문을 작성하세요
"""

# Agent 실행
result = run_agent(my_question)

# 기대 결과: Agent가 여러 도구를 사용하여 종합적인 답변 제공

---

## 정답 확인

모든 문제를 풀었다면, 아래 셀의 주석을 해제하여 정답을 확인할 수 있습니다.

In [ ]:
# 정답 예시 (주석 해제하여 확인)

# # 문제 1
# tool_count = len(AVAILABLE_TOOLS)

# # 문제 2
# for tool in AVAILABLE_TOOLS:
#     print(f"- {tool.name}")

# # 문제 3
# tool = find_tool("get_stock_price")

# # 문제 4
# ticker = "AAPL"
# hist = stock.history(period="1mo")

# # 문제 5
# high = hist['High'].max()
# low = hist['Low'].min()

# # 문제 6
# ma5 = hist['Close'].rolling(window=5).mean()

# # 문제 7
# stock = yf.Ticker(ticker)

# # 문제 8
# @tool

# # 문제 9
# max_calls=3
# tool_name="get_stock_price"
# state.tool_history.add_execution(exec1)

# # 문제 10
# my_question = "Tesla(TSLA)의 기업 정보를 조회하고, 현재가와 50일 이동평균을 비교해줘"